In [26]:
from analysis import collectConnections, print_with_stats
import pandas as pd
import re
import json

In [27]:
to_namedtuple = lambda x: list(x.itertuples())
actions = to_namedtuple(pd.read_csv('../data/all-history-actions.csv'))
actions_df = pd.read_csv('../data/all-history-actions.csv')
findBillNum = re.compile('([HS]\d{1,5})')

In [28]:
actions

[Pandas(Index=0, id='H1', action='Referred to the committee on House Ways and Means', branch='House', date='2021-02-25T16:46:19.933Z'),
 Pandas(Index=1, id='H1', action='Reported, in part, by H4000', branch='House', date='2021-04-15T04:00:00.000Z'),
 Pandas(Index=2, id='H10', action='Referred to the committee on Public Service', branch='House', date='2021-04-13T15:59:37.223Z'),
 Pandas(Index=3, id='H10', action='Senate concurred', branch='Senate', date='2021-04-13T15:59:37.223Z'),
 Pandas(Index=4, id='H10', action='Hearing scheduled for 07/28/2021 from 01:00 PM-04:00 PM in Virtual Hearing', branch='Joint', date='2021-07-22T14:53:10.290Z'),
 Pandas(Index=5, id='H10', action='Hearing rescheduled to 07/28/2021 from 09:30 AM-12:00 PM in Virtual Hearing', branch='Joint', date='2021-07-23T18:22:47.127Z'),
 Pandas(Index=6, id='H10', action='Bill reported favorably by committee and referred to the committee on House Ways and Means', branch='House', date='2021-10-07T18:49:13.353Z'),
 Pandas(Ind

In [29]:
actions_df

,id,action,branch,date
0,H1,Referred to the committee on House Ways and Means,House,2021-02-25T16:46:19.933Z
1,H1,"Reported, in part, by H4000",House,2021-04-15T04:00:00.000Z
2,H10,Referred to the committee on Public Service,House,2021-04-13T15:59:37.223Z
3,H10,Senate concurred,Senate,2021-04-13T15:59:37.223Z
4,H10,Hearing scheduled for 07/28/2021 from 01:00 PM...,Joint,2021-07-22T14:53:10.290Z
...,...,...,...,...
45467,s2729,Recommended new text for H4126,Senate,2022-02-28T05:00:00.000Z
45468,s2729,Rules suspended,Senate,2022-02-28T05:00:00.000Z
45469,s2729,Substituted as a new text for H4126,Senate,2022-02-28T05:00:00.000Z
45470,s2729,See H4126,Senate,2022-02-28T05:00:00.000Z


In [30]:
# TODO: parse action types: "referred to", "hearing scheduled", "reported from"
# TODO: create parse function that converts a raw action string to an array of dicts. Each dict has a 'type' field that identifies its type, and additional attribute fields

referred = re.compile(
    "(?P<type>referred to the committee) on (?P<committee_name>.*)", flags=re.IGNORECASE
)

hearing_scheduled = re.compile(
    "(?P<type>hearing scheduled) for (?P<date>.*) from (?P<time>.*) in (?P<forum>.*)", flags = re.IGNORECASE
)

reported_from = re.compile(
    "(?P<type>reported) from (?P<committee_name>.*)", flags= re.IGNORECASE
)

reading = re.compile(
    "(?P<type>read) (?P<reading>(second|third|second and ordered to a third))", flags= re.IGNORECASE
)

enacted = re.compile(
    "(?P<type>enacted) and (?P<info>.*)", flags= re.IGNORECASE
)

enacted_votes = re.compile(
    "(?P<type>enacted) - (?P<info>.*)", flags= re.IGNORECASE
)


action = actions[0].action
r = referred.match(action)

# actions[0] is a namedtuple. You can convert it to a dict with actions[0]._asdict()
# display(actions[0])
# print(action, r.groupdict())
# print(actions[0]._asdict().keys())

# def parse(action):
#     return [{"type": "reported"}, {"type": "referred", "committee_name": "Technology"}]

In [45]:

action_types = [reported_from, referred]

#creates list of actions from dictionaries
def extract_actions(actions):
    action_dict = action_to_dict(actions)
    extracted_actions = [action['action'] for action in action_dict]
    return extracted_actions


#converts list of named tuples to a list of dictionaries
def action_to_dict(actions):
    action_dict = [action._asdict() for action in actions]
    return action_dict


#creates list of dictionaries of matching action types
def parse_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_actions.append(selected_action.groupdict())
    return selected_actions


def strip_committee(action_dict):
    cleaned_actions = []
    for dict_ in action_dict:
        cleaned_committee = dict_['committee_name'].strip()
        cleaned_dict = {'type': (dict_['type']), 'committee_name': (cleaned_committee)}
        cleaned_actions.append(cleaned_dict)
    return cleaned_actions


def append_enacted_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'info': selected_action_dict['info']}
                    selected_actions.append(selected_dict)
    return selected_actions


def append_committee_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'committee_name': selected_action_dict['committee_name']}
                    selected_actions.append(selected_dict)
    return selected_actions

def append_hearing_actions(action_types, actions_dict):
    selected_actions = []
    for action_type in action_types:
        for action in actions_dict:
            selected_action = action_type.match(action['action'])
            if selected_action:
                    selected_action_dict = selected_action.groupdict()
                    selected_dict = {'id': action['id'], 'branch': action['branch'],
                     'date': action['date'], 'type': selected_action_dict['type'], 'scheduled_date': selected_action_dict['date'], 'time': selected_action_dict['time'], 'forum': selected_action_dict['forum']}
                    selected_actions.append(selected_dict)
    return selected_actions


In [32]:
action_dict = parse_actions(action_types, action_to_dict(actions))

action_dict


[{'type': 'Reported',
  'committee_name': 'the committee on Bonding, Capital Expenditures and State Assets'},
 {'type': 'Reported', 'committee_name': 'the committee on House Rules'},
 {'type': 'Reported',
  'committee_name': 'the committee on House Ways and Means'},
 {'type': 'Reported', 'committee_name': 'the committee on Public Service'},
 {'type': 'Reported', 'committee_name': 'the committee of conference'},
 {'type': 'Reported',
  'committee_name': 'the committee on Consumer Protection and Professional Licensure'},
 {'type': 'Reported',
  'committee_name': 'the committee on Consumer Protection and Professional Licensure'},
 {'type': 'Reported',
  'committee_name': 'the committee on Consumer Protection and Professional Licensure'},
 {'type': 'Reported',
  'committee_name': 'the committee on House Ways and Means'},
 {'type': 'Reported', 'committee_name': 'the committee on Transportation'},
 {'type': 'Reported', 'committee_name': 'the committee on Public Service'},
 {'type': 'Reported

In [33]:
appended_action_types = [enacted, enacted_votes]

append_actions(appended_action_types, action_to_dict(actions))

[{'id': 'H100',
  'branch': 'Senate',
  'date': '2021-06-07T15:26:51.847Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H1917',
  'branch': 'Senate',
  'date': '2022-08-01T06:16:31.113Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2144',
  'branch': 'Senate',
  'date': '2021-11-17T16:07:59.820Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2156',
  'branch': 'Senate',
  'date': '2021-08-02T15:17:50.193Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2166',
  'branch': 'Senate',
  'date': '2021-10-14T19:15:55.493Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2169',
  'branch': 'Senate',
  'date': '2021-12-20T16:10:46.077Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2183',
  'branch': 'Senate',
  'date': '2022-03-24T15:13:45.787Z',
  'type': 'Enacted',
  'info': 'laid before the Governor'},
 {'id': 'H2190',
  'branch': 'Senate',
  '

In [34]:
bill_number = 'H4461'
connections = collectConnections(bill_number)
print_with_stats(connections)

total bills:  8

bill:  H4470
history actions:  20
connections:  {'S2289', 'S2851', 'S2872', 'H4822', 'H3456', 'H4459', 'H4461', 'H4805'}
House 02/16/22 05:00:00 H4461, published as amended
House 02/16/22 05:00:00 Passed to be engrossed - 120 YEAS to 36 NAYS (See YEA and NAY No. 147)
Senate 04/25/22 15:15:37 Read
Senate 04/25/22 04:00:00 Referred on motion (Crighton) to the committee on Senate Rules
Senate 04/28/22 18:05:54 Committee reported that the matter be placed in the Orders of the Day for Thursday, May 5, 2022 with an amendment, inserting in place thereof the text of S2851
Senate 04/28/22 04:00:00 Order relative to subject matter adopted
Senate 04/28/22 04:00:00 Placed in the Orders of the Day for Thursday, May 5, 2022
Senate 05/05/22 20:51:09 Read second
Senate 05/05/22 04:00:00 Amended by striking out all after the enacting clause and insetting in place thereof the text of S2851
Senate 05/05/22 04:00:00 Ordered to a third reading
Senate 05/05/22 04:00:00 Read third
Senate 05/

In [56]:
#exports parsed actions to a JSON file
def write_json(dictionary, str):
    json_object = json.dumps(dictionary, indent=4)
    with open(f"../data/{str}.json", "w") as outfile:
        outfile.write(json_object)

In [57]:
write_json(append_enacted_actions(appended_action_types, action_to_dict(actions)), 'enacted')

In [60]:
action_types = [reported_from, referred]
write_json(append_committee_actions(action_types, action_to_dict(actions)), 'reported-referred')

In [59]:
action_types = [hearing_scheduled]
write_json(append_hearing_actions(action_types, action_to_dict(actions)), 'hearing_scheduled')